# RNN实现

## 导入杰伦的歌词数据

In [1]:
import d2lzh as d2l
import math
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [2]:
print(vocab_size)

1038


## onehot向量输入
onehot向量可以考虑用词嵌入的向量来代替

In [3]:
def to_onehot(X, size):
    return [nd.one_hot(x, size) for x in X.T]
X = nd.arange(10).reshape((2, 5))
print(X)   # [[0. 1. 2. 3. 4.]    一个batch的形状
           # [5. 6. 7. 8. 9.]]
print(X.T)
inputs = to_onehot(X, vocab_size)
inputs
# len(inputs), inputs[0].shape 


[[0. 1. 2. 3. 4.]
 [5. 6. 7. 8. 9.]]
<NDArray 2x5 @cpu(0)>

[[0. 5.]
 [1. 6.]
 [2. 7.]
 [3. 8.]
 [4. 9.]]
<NDArray 5x2 @cpu(0)>


[
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1038 @cpu(0)>, 
 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1038 @cpu(0)>, 
 [[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1038 @cpu(0)>, 
 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1038 @cpu(0)>, 
 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1038 @cpu(0)>]

## 初始化模型

In [4]:
# num_hiddens是超参数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()
print("we will use", ctx, "to train")

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    
    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = nd.zeros(num_hiddens, ctx=ctx)
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    
    #附上梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

we will use cpu(0) to train


## 定义模型

In [7]:
# 返回隐藏状态的和输出
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

- $H_t = \phi(X_tW_{xh} + H_{t-1}W_{hh} + b_n)$
- $O_t = H_tW_{hq} + b_q$

In [8]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vacab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, )

测试输出结果的个数，**第一个时间步的输出层输出的形状**和**隐藏状态的形状**

In [9]:
state = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, (2, 1038), (2, 256))

## 定义预测函数 

In [10]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    outputs = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(nd.array([outputs[-1]], ctx=ctx), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        if t < len(prefix) - 1:
            outputs.append(char_to_idx[prefix[t + 1]])
        else:
            outputs.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in outputs])

测试一下predict_run函数

In [11]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx)

'分开身角酋带站这送爬斜世'

## 裁剪梯度 

为了避免梯度爆炸，要给梯度设定阈值，超过阈值就变1。

公式 : $\min(\frac{\theta}{||g||}, 1) \times g$  这个公式表示，当g的二范数（$\sqrt{g_1^2 + g_2^2}$）小于阈值$\theta$时，g就保持不变，但大于$\theta$时就会对g加上一个缩小乘子$\frac{\theta}{||g||}$

In [12]:
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param in params:
        norm += (param.grad **2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

## 使用困惑度(PPL)

困惑度（PPL）是用来评价语言模型好坏的。困惑度是对交叉熵函数进行指数运算后得到的值。
- 最佳，总是判断正确，分类预测正确概率1，困惑度1.
- 最坏，总是预测错误，分类正确率0，困惑度正无穷
- baseline，预测所有类别的概率都相同，困惑度为类别个数，即vocab_size

有效的模型必须小于类别的个数

## 定义模型训练函数 
与之前章节的模型训练函数相比，不同点：
1. 使用困惑度评价模型
2. 在 迭代 模型参数前 剪裁梯度
3. 在时序数据采用不同采样方法 rnn中的状态初始化也要跟着不同

In [13]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                         vocab_size, ctx, corpus_indices, idx_to_char,
                         char_to_idx, is_random_iter, num_epochs, num_steps,
                         lr, clipping_theta, batch_size, pred_period,
                         pred_len, prefixes
    """
    train_and_predict实际上是：
    1. 进行batch分批
    2. 创建epoch循环
    3. 建立batch遍历，将所有batches遍历一遍
    4. 对于一个batch，使用rnn函数进行前向计算，记录辅助w参数梯度下降计算所用到的其他梯度。然后使用反向传播计算梯度
    """
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:    # 相邻采样
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for X, Y in data_iter:
            if is_random_iter:
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else:
                for s in state:
                    s.detach()
            with autograd.record():
                # [[0. 5.]  经过to_onehot后的形状
                # [1. 6.]
                # [2. 7.]
                # [3. 8.]
                # [4. 9.]]
                inputs = to_onehot(X, vocab_size)
                # 一个inputs中包含的是时间步长个
                outputs, state = rnn(inputs, state, params)
                outputs = nd.concat(*outputs, dim=0)
                
                y = Y.T.reshape((-1, ))
                
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params,clipping_theta, ctx)
            d2l.sgd(params, lr, 1)
            l_sum += l.asscalar() * y.size  # 直接转化为了对softmax运算的求和的
            n += y.size
            
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                    prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx
                    ))

训练

In [14]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2   # e-2 是 10^{-2}
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开'] 

train_and_predict_rnn(
    rnn, get_params, init_rnn_state,
    num_hiddens, vocab_size, ctx, corpus_indices,
    idx_to_char, char_to_idx, True, num_epochs, 
    num_steps, lr, clipping_theta, batch_size, 
    pred_period, pred_len, prefixes
)

epoch 50, perplexity 69.552116, time 4.23 sec
 - 分开 我不要再想你 不知我不想你 不知我不 你不 我不要再想 我不要再想 我不要再想 我不要再想 我不要
 - 不分开 我不要再想你 不知我不想你 不知我不 你不 我不要再想 我不要再想 我不要再想 我不要再想 我不要
epoch 100, perplexity 10.109862, time 1.98 sec
 - 分开 用一定美 快暖了外 恨果己空 恨我不起 却你说口 如果了空  我不要 静怎么的字活 我知你 你爱我
 - 不分开 爱要我 别怪我 想你怎么单！ 折这么(客  我爱道这里很久 我想要 想你眼睛看着我 泪开的我 你的
epoch 150, perplexity 2.871562, time 4.60 sec
 - 分开 用色啦烛 温水了空屋人占卜 她说下午三点阳光射by堂 我的世界已狂的暴言 纪录第一次遇见的你（Ja
 - 不分开期 然伤将过的牛肉 我都店的有模有样 什么兵器最喜欢 双截棍柔中带染 想要你枪跟故 干什么(客) 瞎
epoch 200, perplexity 1.605749, time 1.97 sec
 - 分开 用是心不的 静晶就什么抢了)在窝 那子风化千年的誓言 一切又重演 祭在西元前 爱在它不前 这蝪拽著
 - 不分开 已经 失去意义 戒指在哭泣 静静躺在抽屉 它所拥有的只剩下回忆 草原上 你时我 一九四三 在漠忆 
epoch 250, perplexity 1.308990, time 4.20 sec
 - 分开 用我心口你 我听说你球在战争就就会换来和 只是我怕眼 让话开你已 有种不要走 三底三什么 懂不懂篮
 - 不分开 已经 失去意义 戒指在哭泣 静静躺在抽屉 它所拥有的只剩下回忆 还一定钟身时香香始摩远 一定的叹有


## 总结

- 小组件
    1. to_onehot()输入数据变为onehot，大概是因为网络是由0，1激活态构成
    2. get_params()获得初始化的参数
    3. init_rnn_state()初始化rnn状态
- 中组件，调用小组件
    1. rnn()
    2. predict_rnn()
- 大组件，整合中组件和小组件
    1. train_and_predict()    